In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures

In [2]:
X_train = pd.read_csv('ds6-league-of-legends/participants_train.csv')
X_test = pd.read_csv('ds6-league-of-legends/participants_test.csv')
y_train = pd.read_csv('ds6-league-of-legends/train_winners.csv')
submission = pd.read_csv('ds6-league-of-legends/sample_submission.csv')
champion_mastery = pd.read_csv('ds6-league-of-legends/champion_mastery.csv')
Position = pd.read_csv('ds6-league-of-legends/teamPositions.csv')
X_train_timeline = pd.read_csv('ds6-league-of-legends/train_timeline_values.csv')
Y_test_timeline = pd.read_csv('ds6-league-of-legends/test_timeline_values.csv')
champions = pd.read_json('ds6-league-of-legends/champion.json')

In [3]:
#creating a copy of the X data to avoid corruption in the dataset
X_train_original = X_train
y_train_original = y_train
X_test_original = X_test

# this is to extract the columns that is needed for training
y_train_timeline = y_train['winner']

In [4]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_train = pd.merge(X_train_original, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_train = X_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_train = (
    X_train
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

X_train = X_train.groupby('matchId')[variables].sum().reset_index(drop = True)


X_train

,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,1392,60.0,31685,35932,605,988,12308,25263,43020,1318,1314,69
1,1137,58.0,32478,35798,571,974,12050,20432,39285,2231,2231,68
2,1277,49.0,33721,34304,609,1083,12413,29570,47450,678,678,66
3,1114,59.0,34392,33758,649,1061,12000,29486,47320,1910,1910,66
4,1099,49.0,32161,35458,604,1036,12109,23707,41644,963,963,69
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1670,60.0,33882,35780,594,1040,12311,28536,44465,817,816,70
7996,1406,58.0,32553,35225,741,999,12477,27303,43455,1324,1324,67
7997,1031,58.0,34316,32811,654,1090,11858,29643,42561,2964,2963,67
7998,1854,58.0,33854,34507,555,964,12652,26260,39305,558,557,68


**The following code calculate the NN classification for league of leguends**

In [5]:
# Divides the data into training and testing 
X_train, X_test, Y_train, Y_test = train_test_split(X_train, y_train_timeline, stratify = y_train_timeline, random_state = 345)

In [6]:
y_train_timeline = y_train_timeline.sample(n=6000, random_state=345)

In [7]:
X_test = X_test.sample(n=2000, random_state=42)
Y_test_timeline = Y_test_timeline.sample(n=2000, random_state=42)

In [8]:
pipe = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(hidden_layer_sizes = (2,),
                             activation = 'tanh',
                             max_iter = 10000))
    ]
)

In [9]:
pipe.fit(X_train, y_train_timeline)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('nn',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(2,),
                               max_iter=10000))])

In [10]:
y_pred = pipe.predict(X_test)
y_pred

array([100, 100, 100, ..., 100, 100, 100])

In [11]:
accuracy_score(Y_test, pipe.predict(X_test))

0.509

In [12]:
submission['winner'] = y_pred
submission.head()

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,100
4,8004,100


In [13]:
#submission.to_csv('ds6-league-of-legends/submission.csv', index = False)

**Testing something different since the predictions are low still**

In [14]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_test = pd.merge(X_test_original, Y_test_timeline, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_test = (
    X_test
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

X_test = X_test.groupby('matchId')[variables].sum().reset_index(drop = True)

# correcting an error

The error message "ValueError: Multioutput target data is not supported with label binarization" is typically raised when you are trying to binarize multi-output targets using the LabelBinarizer class in pandas.

The LabelBinarizer class is used to binarize categorical target variables into binary indicator variables. However, it only supports single-output targets with one dimension. Multi-output targets with multiple dimensions are not supported.

To fix this error, you need to use a different approach to binarize multi-output targets. One possible solution is to use the MultiLabelBinarizer class instead of the LabelBinarizer class.

In [15]:
data = pd.json_normalize(champions['data'])
data

,version,id,key,name,title,blurb,tags,partype,info.attack,info.defense,...,stats.hpregen,stats.hpregenperlevel,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed
0,13.5.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,8,4,...,3.00,1.00,0.00,0.00,0,0,60,5.00,2.500,0.651
1,13.5.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"[Mage, Assassin]",Mana,3,4,...,2.50,0.60,8.00,0.80,0,0,53,3.00,2.000,0.668
2,13.5.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,5,3,...,9.00,0.90,50.00,0.00,0,0,62,3.30,3.200,0.625
3,13.5.1,Akshan,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","[Marksman, Assassin]",Mana,0,0,...,3.75,0.65,8.20,0.70,0,0,52,3.50,4.000,0.638
4,13.5.1,Alistar,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,6,9,...,8.50,0.85,8.50,0.80,0,0,62,3.75,2.125,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,13.5.1,Zeri,221,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",[Marksman],Mana,8,5,...,3.25,0.70,6.00,0.80,0,0,53,1.30,2.000,0.658
158,13.5.1,Ziggs,115,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",[Mage],Mana,2,4,...,6.50,0.60,8.00,0.80,0,0,54,3.10,2.000,0.656
159,13.5.1,Zilean,26,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","[Support, Mage]",Mana,2,5,...,5.50,0.50,11.35,0.80,0,0,52,3.00,2.130,0.625
160,13.5.1,Zoe,142,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","[Mage, Support]",Mana,1,7,...,6.50,0.60,8.00,0.65,0,0,58,3.30,2.500,0.625


In [16]:
data['key'] = data['key'].astype(int)
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl',
             'info.attack', 'info.defense', 'stats.hpregen', 'stats.mpregen', 'stats.attackdamage', 'stats.attackspeed']

X = pd.merge(X_train_original, data, how='inner', left_on='championId', right_on='key')

X = pd.merge(X, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X = pd.merge(X, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X = X.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X = (
    X
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum',
          'stats.attackspeed': 'sum',
          'info.attack': 'sum',
          'stats.mpregen': 'sum',
          'stats.attackdamage': 'sum',
          'info.defense': 'sum',
          'stats.hpregen': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

X = X.groupby('matchId')[variables].sum().reset_index(drop = True)

In [17]:
y = y_train_timeline

y = y.sample(n=8000, replace=True)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [19]:
pipe = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(hidden_layer_sizes = (2,),
                             activation = 'tanh',
                             max_iter = 10000))
    ]
)

In [20]:
pipe.fit(X, y)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('nn',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(2,),
                               max_iter=10000))])

In [21]:
y_pred = pipe.predict(X_test)
y_pred

array([100, 100, 100, ..., 100, 100, 100])

In [22]:
accuracy_score(y, pipe.predict(X))

0.50475